In [ ]:
import random
import numpy as np
import torch
from torch import nn, optim

from models import *

/home/local/VANDERBILT/lij112/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = Mamba2Multitask(
    input_dim=46,
    d_model=256,
    d_state=64,
    d_conv=4,
    expand=2,
    num_layers=4,
).cuda()
opt   = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

B, L, D = 8, 8640, 46
chunk_size = 1024
x = torch.randn(B, L, D, device="cuda")
dir_tar = torch.randn(B, 1, device="cuda")
reg_tar = torch.randn(B, 3, device="cuda")

opt.zero_grad()
total_loss = 0.0
for start in range(0, L, chunk_size):
    end = start + chunk_size
    x_chunk = x[:, start:end]

    dir_logits, reg_out = model(x_chunk)
    loss = loss_fn(dir_logits, dir_tar) + loss_fn(reg_out, reg_tar)
    loss.backward()

    opt.step()
    opt.zero_grad()
    
    for p in model.parameters():
        if p.grad is not None:
            p.grad.detach_()
            p.grad.zero_()

    total_loss += loss.item()
print("Epoch loss:", total_loss)

Epoch loss: 21.742488145828247
